<a href="https://colab.research.google.com/github/ashwanimsajeev/AI/blob/main/next_word_pred_lstm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
uploaded = files.upload()


Saving story_text.txt to story_text (1).txt


In [ ]:
import os
print(os.listdir())


['.config', 'story_text (1).txt', 'story_text.txt', 'sample_data']


In [ ]:
data_path = "story_text.txt"  # Now it's in current directory
with open(data_path, "r", encoding="utf-8") as file:
    text = file.read().lower()


In [ ]:
with open("story_text.txt", "r", encoding="utf-8") as file:
    text = file.read().lower()

# Use only the first 100,000 characters (adjust as needed)
text = text[:100000]


In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import LSTM, Dense, Embedding, Dropout
from keras.utils import to_categorical
import os

# STEP 1: Load your downloaded text dataset (.txt file)
data_path = "story_text.txt"  # Change this to your actual file name
with open(data_path, "r", encoding="utf-8") as file:
    text = file.read().lower()

# STEP 2: Tokenize the text
tokenizer = Tokenizer()
tokenizer.fit_on_texts([text])
total_words = len(tokenizer.word_index) + 1

# STEP 3: Create input sequences using n-gram technique (with memory limit)
input_sequences = []
for line in text.split(". "):
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i + 1]
        input_sequences.append(n_gram_sequence)

    # Limit to avoid memory crash
    if len(input_sequences) > 10000:
        break


# STEP 4: Pad sequences
max_sequence_length = max(len(seq) for seq in input_sequences)
input_sequences = pad_sequences(input_sequences, maxlen=max_sequence_length, padding='pre')

# STEP 5: Split into features (X) and label (y)
X, y = input_sequences[:, :-1], input_sequences[:, -1]
y = to_categorical(y, num_classes=total_words)

# STEP 6: Build the LSTM Model
model = Sequential()
model.add(Embedding(total_words, 100, input_length=max_sequence_length - 1))
model.add(LSTM(150, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(100))
model.add(Dense(100, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(total_words, activation='softmax'))

# STEP 7: Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# STEP 8: Train the model
model.fit(X, y, epochs=30, verbose=1, batch_size=128)

# Optional: Save the model
model.save("lstm_next_word_model.h5")

# STEP 9: Function to Predict Next N Words
def predict_next_words(model, tokenizer, input_text, n_words):
    for _ in range(n_words):
        token_list = tokenizer.texts_to_sequences([input_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_length - 1, padding='pre')
        predicted = model.predict(token_list, verbose=0)
        output_word = tokenizer.index_word.get(np.argmax(predicted), '')
        input_text += " " + output_word
    return input_text

# STEP 10: Use the model to predict the next words
input_text = "machine learning is"   # Change this to your sentence
n_words = 5  # Number of next words to predict
predicted_text = predict_next_words(model, tokenizer, input_text, n_words)
print("Predicted Text:", predicted_text)


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/30
79/79 ━━━━━━━━━━━━━━━━━━━━ 166s 2s/step - accuracy: 0.0625 - loss: 9.1829
Epoch 2/30
79/79 ━━━━━━━━━━━━━━━━━━━━ 159s 2s/step - accuracy: 0.0785 - loss: 6.1917
Epoch 3/30
79/79 ━━━━━━━━━━━━━━━━━━━━ 201s 2s/step - accuracy: 0.0777 - loss: 6.0485
Epoch 4/30
79/79 ━━━━━━━━━━━━━━━━━━━━ 202s 2s/step - accuracy: 0.0775 - loss: 5.9758
Epoch 5/30
79/79 ━━━━━━━━━━━━━━━━━━━━ 201s 2s/step - accuracy: 0.0804 - loss: 5.9529
Epoch 6/30
79/79 ━━━━━━━━━━━━━━━━━━━━ 204s 2s/step - accuracy: 0.0771 - loss: 5.9409
Epoch 7/30
79/79 ━━━━━━━━━━━━━━━━━━━━ 203s 2s/step - accuracy: 0.0779 - loss: 5.9368
Epoch 8/30
79/79 ━━━━━━━━━━━━━━━━━━━━ 202s 2s/step - accuracy: 0.0778 - loss: 5.9235
Epoch 9/30
79/79 ━━━━━━━━━━━━━━━━━━━━ 201s 2s/step - accuracy: 0.0786 - loss: 5.8533
Epoch 10/30
79/79 ━━━━━━━━━━━━━━━━━━━━ 160s 2s/step - accuracy: 0.0777 - loss: 5.7387
Epoch 11/30
79/79 ━━━━━━━━━━━━━━━━━━━━ 202s 2s/step - accuracy: 0.0789 - loss: 5.6453
Epoch 12/30
79/79 ━━━━━━━━━━━━━━━━━━━━ 201s 2s/step - accuracy:

Predicted Text: machine learning is a red rose and he
